# Dinâmica simbólica em Python

Este notebook demonstra como calcular as matrizes **M**, **C**, **H** e **G** para um robô de cadeia aberta usando os módulos Python criados a partir do projeto em MATLAB. A estrutura separa o código em vários arquivos `.py` e o notebook atua apenas como orquestrador.

### 1. Clonar o repositório

Você pode clonar seu repositório usando o comando `!git clone` no Colab.

In [1]:
# Substitua esta URL pela URL do seu repositório Git
repo_url = "https://github.com/danisilva1110-spec/PFPython.git"

# Nome da pasta de destino (derivado automaticamente da URL)
repo_dir = repo_url.rstrip("/").split("/")[-1]
if repo_dir.endswith(".git"):
    repo_dir = repo_dir[:-4]

import sys
from pathlib import Path

current_repo = Path.cwd()
local_package = current_repo / "robot_dynamics"

# Se o notebook já estiver dentro do repositório, use-o diretamente
if local_package.exists():
    repo_path = current_repo
else:
    repo_path = current_repo / repo_dir
    if not repo_path.exists():
        # Clone o repositório apenas se ele ainda não existir
        !git clone {repo_url} {repo_dir}

# Garanta que o repositório esteja no sys.path para que os módulos sejam encontrados
sys.path.insert(0, str(repo_path.resolve()))

Cloning into 'PFPython'...
remote: Enumerating objects: 312, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 312 (delta 5), reused 2 (delta 2), pack-reused 296 (from 1)
Receiving objects: 100% (312/312), 413.72 KiB | 980.00 KiB/s, done.
Resolving deltas: 100% (170/170), done.


In [2]:
from sympy import simplify, symbols

from robot_dynamics import (
    forward_kinematics,
    inertia_matrix,
    coriolis_matrix,
    centripetal_vector,
    gravity_vector,
    kinetic_energy,
    planar_two_dof,
)

### 2. Listar o conteúdo do repositório clonado
Após clonar (ou ao usar este notebook diretamente dentro do repositório), a variável `repo_path` aponta para a pasta onde o código está. Você pode inspecionar os arquivos com o comando abaixo.


In [3]:
# Substitua 'repo_path' pelo caminho do repositório se você tiver alterado as variáveis acima
!ls {repo_path}

main.ipynb  README.md  robot_dynamics


## Definição do robô

Criamos um manipulador planar de 2 DOF com parâmetros de Denavit-Hartenberg padrão e propriedades inerciais simbólicas.

In [4]:
(links, q, qd, g_vec) = planar_two_dof()
q1, q2 = q
dq1, dq2 = qd

# Caso seja necessário ajustar os parâmetros simbólicos, basta usar subs nos objetos retornados.

## Cinemática direta

As transformadas homogêneas de cada elo são acumuladas com base nos parâmetros DH fornecidos.

In [5]:
transforms = forward_kinematics(links, q)
transforms

[Matrix([
 [cos(q1), -sin(q1), 0, l1*cos(q1)],
 [sin(q1),  cos(q1), 0, l1*sin(q1)],
 [      0,        0, 1,          0],
 [      0,        0, 0,          1]]),
 Matrix([
 [-sin(q1)*sin(q2) + cos(q1)*cos(q2), -sin(q1)*cos(q2) - sin(q2)*cos(q1), 0, l1*cos(q1) - l2*sin(q1)*sin(q2) + l2*cos(q1)*cos(q2)],
 [ sin(q1)*cos(q2) + sin(q2)*cos(q1), -sin(q1)*sin(q2) + cos(q1)*cos(q2), 0, l1*sin(q1) + l2*sin(q1)*cos(q2) + l2*sin(q2)*cos(q1)],
 [                                 0,                                  0, 1,                                                    0],
 [                                 0,                                  0, 0,                                                    1]])]

## Matrizes da dinâmica

A partir da energia cinética e potencial calculamos **M(q)**, **C(q,\dot q)**, o vetor centrípeto **H = C\,\dot q** e o vetor de gravidade **G(q)**.

In [6]:
M = simplify(inertia_matrix(links, q, qd))
C = simplify(coriolis_matrix(links, q, qd))
H = simplify(centripetal_vector(C, qd))
G = simplify(gravity_vector(links, q, g_vec))

M, C, H, G

(Matrix([
 [1.0*I1zz + 1.0*I2zz + 2.25*l1**2*m1 + 0.25*m2*(4*l1**2 + 12*l1*l2*cos(q2) + 9*l2**2), 1.0*I2zz + 0.75*l2*m2*(2*l1*cos(q2) + 3*l2)],
 [                                         1.0*I2zz + 0.75*l2*m2*(2*l1*cos(q2) + 3*l2),                    1.0*I2zz + 2.25*l2**2*m2]]),
 Matrix([
 [-1.5*dq2*l1*l2*m2*sin(q2), 1.5*l1*l2*m2*(-dq1 - dq2)*sin(q2)],
 [ 1.5*dq1*l1*l2*m2*sin(q2),                                 0]]),
 Matrix([
 [1.5*dq2*l1*l2*m2*(-2*dq1 - dq2)*sin(q2)],
 [            1.5*dq1**2*l1*l2*m2*sin(q2)]]),
 Matrix([
 [g*(3*l1*m1*cos(q1) + 2*l1*m2*cos(q1) + 3*l2*m2*cos(q1 + q2))/2],
 [                                      3*g*l2*m2*cos(q1 + q2)/2]]))

## Fluxo com matrizes (excentricidade, tensores e eixos)

Se você já possui as tabelas de Denavit–Hartenberg, os vetores de massa, as excentricidades (offsets do centro de massa) e os tensores de inércia simetrizados, pode montar tudo de uma vez usando `equations_of_motion_from_matrices`. O exemplo abaixo reutiliza o manipulador planar de 2 DOF e deixa explícito o eixo de cada junta.


In [ ]:
from sympy import Matrix, symbols

from robot_dynamics import equations_of_motion_from_matrices

# Símbolos do exemplo
q1, q2, dq1, dq2, l1, l2, m1, m2, g, I1zz, I2zz = symbols("q1 q2 dq1 dq2 l1 l2 m1 m2 g I1zz I2zz")

dh_params = [
    (0, 0, 0, q1),          # linha DH do elo 1
    (l1, 0, 0, q2),        # linha DH do elo 2
]
joint_types = ["R", "R"]
masses = [m1, m2]
excentricities = [
    (l1 / 2, 0, 0),        # COM do elo 1
    (l2 / 2, 0, 0),        # COM do elo 2
]
inertia_tensors = [
    [[0, 0, 0], [0, 0, 0], [0, 0, I1zz]],
    [[0, 0, 0], [0, 0, 0], [0, 0, I2zz]],
]
axis_orders = ["z", "z"]  # eixos de movimento de cada junta (podem ser x/y/z)

gravity = Matrix([0, -g, 0])
resultado = equations_of_motion_from_matrices(
    dh_params,
    joint_types,
    masses,
    excentricities,
    inertia_tensors,
    q=(q1, q2),
    qd=(dq1, dq2),
    gravity=gravity,
    axis_orders=axis_orders,
)

M = resultado["M"]
C = resultado["C"]
H = resultado["H"]
G = resultado["G"]
M, C, H, G


> Dica: se alguma junta translada ou gira em outro eixo, basta trocar a entrada correspondente em `axis_orders` por `"x"` ou `"y"` (aceitando também prefixos `R`/`P` para legibilidade). Os tensores em `inertia_tensors` são simetrizados automaticamente antes do uso.


## Variáveis automáticas e máscara 0/1

`equations_of_motion_from_order` gera `q`/`qd` a partir da ordem de juntas (Dx/Dy/Dz/x/y/z) e aceita um `active_mask` para desligar DOF sem mexer nas tabelas. O exemplo abaixo segue a ordem do UVMS do MATLAB e usa máscara para travar as três translações do ROV.


In [ ]:
from sympy import Matrix, symbols
from robot_dynamics import equations_of_motion_from_order

axis_order = ["Dx", "Dy", "Dz", "z", "y", "x", "z", "y", "y", "z", "y", "z"]
active_mask = [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]

# DH placeholders (alinhados com axis_order)
dh_params = [(0, 0, 0, 0) for _ in axis_order]

# Excentricidades do UVMS (mesmo formato do MATLAB)
matriz_excentricidades = [
    (0, 0, 0),
    (0, 0, 0),
    (0, 0, 0),
    (0, 0, 0),
    (0, 0, 0),
    (0, 0, 0),
    (0, 0, 1),
    (1, 0, 0),
    (1, 0, 0),
    (1, 0, 0),
    (0, 0, 0),
    (0, 0, 1),
]

# Tensores de inércia identidade (placeholders)
I3 = Matrix.eye(3)
inercia = [I3] * len(axis_order)

m = symbols("m1:13")
g = symbols("g")
gravity = Matrix([0, 0, -g])

resultado_mask = equations_of_motion_from_order(
    dh_params,
    axis_order,
    m,
    matriz_excentricidades,
    inercia,
    gravity=gravity,
    active_mask=active_mask,
)

resultado_mask["M"]
